In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from skimage import io, transform
import utils_xy
from torchvision import transforms, utils
import numpy as np
from PIL import Image

In [3]:
gpu_id = 1
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

device = torch.device('cuda')
print (device)

cuda


### I. Load the image and mask
There are 43 images and 43 masks respectively, for each image, the size is 1024x1024x3, for each mask, the size is 1024x1024

In [4]:
img_dir = '/home/xiaoyu/BMMC/BMMCdata'
mask_dir = '/home/xiaoyu/BMMC/BMMCmasks'

img_list = sorted(os.listdir(img_dir))
mask_list = sorted(os.listdir(mask_dir))

print("Training images numbers: "+str(len(img_list)))
print("Training mask Images numbers:"+str(len(mask_list)))

Training images numbers: 43
Training mask Images numbers:43


### Find the unique color in mask
The unique color in mask is 85, 170, 255, so we can consider there are 3 classes in mask

In [5]:
colors = torch.tensor([])
for i in range(len(img_list)):
    img_arr = io.imread(os.path.join(img_dir, img_list[i]))
    img_tensor = torch.from_numpy(np.uint8(img_arr))
    img_tensor = img_tensor.permute(2,0,1)
    
    mask_str = mask_list[i]
    mask_arr = io.imread(os.path.join(mask_dir, mask_str))
    mask_tensor = torch.from_numpy(mask_arr)
    unique_color = torch.unique(mask_tensor).type(torch.FloatTensor)
    colors = torch.cat((colors,unique_color))
colors = torch.unique(colors)
print(colors)

tensor([ 85., 170., 255.])


### Define the traindataset

In [6]:
class TrainDataset(Dataset):
    
    def __init__(self, img_dir, mask_dir, transform=None):
      
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        
        self.mapping = {
            85: 0,
            170: 1,
            255: 2
        }
        
    def mask_to_class(self, mask):
        for k in self.mapping:
            mask[mask==k] = self.mapping[k]
        return mask
        
    def __len__(self):
        return len(os.listdir(self.img_dir))
    
    def __getitem__(self, idx):
        
        img_list = sorted(os.listdir(img_dir))
        mask_list = sorted(os.listdir(mask_dir))
        
        img_str = img_list[idx]
        img_arr = io.imread(os.path.join(img_dir, img_str))
        img_tensor = torch.from_numpy(np.uint8(img_arr))
        img_tensor = img_tensor.permute(2,0,1)
        
        mask_str = mask_list[idx]
        mask_arr = io.imread(os.path.join(mask_dir, mask_str))
        mask_tensor = torch.from_numpy(mask_arr)
        
        mask_class = self.mask_to_class(mask_tensor)
        
        sample = {'image':img_tensor, 'mask':mask_class}
        
        if self.transform:
            sample = self.transform(sample)
        return sample

In [6]:
# TrainDataset Test
traindata = TrainDataset(img_dir = img_dir, mask_dir = mask_dir)
print(traindata[0]['image'].shape)
print(traindata[0]['mask'].shape)

torch.Size([3, 1024, 1024])
torch.Size([1024, 1024])


### Define the dataloader

In [16]:
dataloader = DataLoader(traindata, batch_size=1, shuffle = False, num_workers = 4)

#### There are 3 classes in total, for every pixel in image, there is a class assigned in ground truth mask

### II. Define the network

In [19]:
class MyNet(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, stride):
        super(MyNet, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, padding, stride)
        
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding, stride)
        
    def forward(self, x):
        x = self.conv1(x)
#         x = conv2(x)
        return x

In [56]:
mynet = MyNet(in_channels = 3, out_channels = 12, kernel_size = 5, stride = 2, padding = 1)

In [57]:
print(mynet)

MyNet(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
)


In [58]:
mynet_params = list(mynet.parameters())
print(mynet_params)

[Parameter containing:
tensor([[[[ 0.0795, -0.0976, -0.0133,  0.0555,  0.0326],
          [ 0.0297, -0.0397, -0.1002, -0.0518,  0.0422],
          [-0.0729,  0.0295,  0.0272,  0.0671,  0.0111],
          [-0.0918, -0.1007, -0.0795, -0.0767,  0.0880],
          [-0.0053,  0.0968,  0.0829, -0.0126, -0.0127]],

         [[ 0.0306,  0.0734, -0.0526, -0.0868, -0.0198],
          [ 0.1075, -0.0404, -0.0423,  0.1097, -0.0553],
          [-0.0541,  0.0497, -0.0396,  0.0013,  0.0855],
          [-0.0440, -0.0237,  0.0382, -0.0048, -0.0561],
          [ 0.0824,  0.0297, -0.0942, -0.0950,  0.0884]],

         [[ 0.0599, -0.0888,  0.0803,  0.0697, -0.0684],
          [ 0.0072, -0.0635,  0.0683,  0.1062, -0.0504],
          [ 0.0297,  0.1040, -0.0433, -0.0726, -0.0607],
          [-0.0647,  0.0513,  0.0313, -0.0224,  0.0392],
          [ 0.0619,  0.0510, -0.0281,  0.1063,  0.0181]]],


        [[[ 0.0449, -0.0663,  0.0083, -0.0125,  0.1106],
          [-0.0403,  0.0553, -0.1018,  0.0447,  0.0554],


In [59]:
print("The length of the mynet parameter is: ")
print(len(mynet_params))

The length of the mynet parameter is: 
2


In [60]:
print("The conv1's weight: ")
print(mynet_params[0].size()) 

The conv1's weight: 
torch.Size([12, 3, 5, 5])


In [61]:
current_device = torch.cuda.current_device()
torch.cuda.device(current_device)

In [62]:
torch.cuda.get_device_name(0)

'GeForce GTX 1080 Ti'

In [63]:
mynet = mynet.to(device)

In [64]:
criterion = nn.NLLLoss()
mylr = 0.05

In [65]:
for epoch in range(20):
    # loop over the dataset 1 times
    running_loss = 0.0
    if epoch%5==4:
        mylr = mylr/5
    optimizer = optim.SGD(mynet.parameters(), lr=mylr)
    
    for i_batch, sample_batched in enumerate(dataloader):
        
        #get the inputs
        inputs, mask = sample_batched['image'], sample_batched['mask']
        
        inputs_float = inputs.type(torch.FloatTensor)
        mask_float = mask.type(torch.FloatTensor)
        
        #zero the parameter gradients
        optimizer.zero_grad()
        
        #forward + backward +optimize
        outputs = mynet(inputs_float.cuda())
        print(outputs.shape)
        
        outputs_flat = outputs.view(-1)
        labels_flat = labels_float.view(-1)
    
        # Define the loss
        loss = criterion(outputs_flat, labels_flat.cuda())      
        loss.backward()
        optimizer.step()
        
        #print statistics
        running_loss += loss.item()
        if i_batch%20 == 19: 
            # print every 20 mini-batchs
            print('[%d, %5f] loss: %.3f'%(epoch +1, i_batch+1, running_loss/20))
            running_loss = 0.0
print('Finish Training')

torch.Size([1, 12, 1024, 1024])


ValueError: Expected 2 or more dimensions (got 1)